In [1]:
# Add files to sys.path
from pathlib import Path
import sys,os
this_path = None
try:    # For .py
    this_path = str(os.path.dirname(os.path.abspath(__file__))) #str(Path().absolute())+"/" # str(os.path.dirname(__file__))
except: # For .ipynb
    this_path = str(Path().absolute())+"/" #str(Path().absolute())+"/" # str(os.path.dirname(__file__))
print("File Path:", this_path)
sys.path.append(os.path.join(this_path, "kinemats"))

# Import classes
import utils  # Utils for generation of files and paths
import quaternion_math

from plotter.ts_visualization import *
import ts_processing
import ts_classification

# Import data science libs
import numpy as np
import pandas as pd

import matplotlib
#matplotlib.rcParams['text.usetex'] = True
#%matplotlib inline
import matplotlib.pyplot as plt

# Feature based classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, StratifiedKFold


File Path: C:\Users\User\Downloads\head-motion-classification/


---
# SETUP

In [2]:
# CONSTANTS
import experiment_config
from experiment_config import Datasets, DataRepresentation, Classifiers
from ts_classification import EnumDistMetrics

# All the files generated from this notebook are in a subfolder with this name
NOTEBOOK_SUBFOLDER_NAME = '2_FeatureBasedClassifiers/'

# Filenames of created files from this script
FILENAME_DATASET_QUATERNION = str(experiment_config.PREFIX_DATASET+str(DataRepresentation.Quaternion))      # generates "dataset_quaternion"
FILENAME_DATASET_EULER = str(experiment_config.PREFIX_DATASET+str(DataRepresentation.Euler))
FILENAME_DATASET_YAW = str(experiment_config.PREFIX_DATASET+str(DataRepresentation.Yaw))

#### NOTE: This dictionary is reassigned later in the code whenever the datasets are generated.
DICT_DATA = {
    DataRepresentation.Quaternion:  None,
    DataRepresentation.Euler:       None,
    DataRepresentation.Yaw:         None,
    DataRepresentation.All:         None,
}
# Dictionary to convert a datarepresentation into a num - To be stored in the numpy array for results
DICT_DATA_TO_NUM = { k:i for i,k in enumerate(DICT_DATA.keys())}

### Headers of tabular data for feature-based classifiers
SUFFIX_DIFFERENTIAL_TRANSFORMATIONS = ["raw","vel","acc"]

# Combination of each dimension with each transformation
HEADER_QUAT = [ h+"_"+s for s in SUFFIX_DIFFERENTIAL_TRANSFORMATIONS for h in ["qw","qi","qj","qk"] ]
HEADER_EULER = [ h+"_"+s for s in SUFFIX_DIFFERENTIAL_TRANSFORMATIONS for h in ["yaw","pitch","roll"] ]
HEADER_YAW = [ h+"_"+s for s in SUFFIX_DIFFERENTIAL_TRANSFORMATIONS for h in ["yaw"] ]
HEADER_ALL = HEADER_QUAT + HEADER_EULER
print(f"Example header {HEADER_QUAT}")

# Setup of overlapping windows to extract features from time series
SLIDING_WINDOW_WIDTH_SECS = 1
SLIDING_WINDOW_OVERLAP_SECS = 0 # Not used!!

#### Classification methods to apply.
DICT_CLASSIFIERS = {
    Classifiers.MLP:    MLPClassifier(random_state=1, max_iter=300),
    Classifiers.KNN:    KNeighborsClassifier(n_neighbors=experiment_config.KNN_N_NEIGH),
    Classifiers.DT:     DecisionTreeClassifier(max_depth=experiment_config.DT_MAX_DEPTH, criterion='entropy', random_state=experiment_config.MC_RANDOM_SEED),
    Classifiers.RF:     RandomForestClassifier(n_estimators=experiment_config.RF_N_ESTIMATORS, max_depth=experiment_config.RF_MAX_DEPTH, criterion='entropy', random_state=experiment_config.MC_RANDOM_SEED),
    Classifiers.GBM:    GradientBoostingClassifier(n_estimators=experiment_config.GBM_N_ESTIMATORS, max_depth=experiment_config.GBM_MAX_DEPTH, criterion='friedman_mse', random_state=experiment_config.MC_RANDOM_SEED)
}

## K-Fold partition
N_SPLITS_CV = experiment_config.CV_NUM_FOLDS # Number of folds for Cross-validation
strat_KFold = StratifiedKFold(n_splits=experiment_config.CV_NUM_FOLDS, random_state=experiment_config.MC_RANDOM_SEED, shuffle=True)

# Scoring parameters: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
SCORING_METRICS = ["accuracy", "precision_macro", "recall_macro", "f1_macro"]

Example header ['qw_raw', 'qi_raw', 'qj_raw', 'qk_raw', 'qw_vel', 'qi_vel', 'qj_vel', 'qk_vel', 'qw_acc', 'qi_acc', 'qj_acc', 'qk_acc']


---
# UTILITY FUNCTIONS

Generate paths to write output files

In [3]:
STR_DATASET = str(experiment_config.DATASET_MAIN)+"/"
print(experiment_config.DATASET_MAIN)
def gen_path_plot(filename):
    # Generates full paths for PLOTS just by specifying a name
    return utils.generate_complete_path(filename, \
                                        main_folder=experiment_config.PLOT_FOLDER, \
                                        subfolders=STR_DATASET+NOTEBOOK_SUBFOLDER_NAME, \
                                        file_extension=experiment_config.IMG_FORMAT, save_files=experiment_config.EXPORT_PLOTS)

def gen_path_temp(filename, subfolders="", extension=experiment_config.TEMP_FORMAT):
    # Generates full paths for TEMP FILES just by specifying a name
    return utils.generate_complete_path(filename, \
                                        main_folder=experiment_config.TEMP_FOLDER, \
                                        subfolders=STR_DATASET+subfolders, \
                                        file_extension=extension)

def gen_path_results(filename, subfolders="", extension=""):
    # Generates full paths for RESULTS FILES (like pandas dataframes)
    return utils.generate_complete_path(filename, \
                                        main_folder=experiment_config.RESULTS_FOLDER, \
                                        subfolders=STR_DATASET+NOTEBOOK_SUBFOLDER_NAME+subfolders, \
                                        file_extension=extension)

Tsinghua


# DATASETS: Load and preprocess

Datasets are adapted to have the reference right-hand coordinate system used: front=1, left=j, up=k

In [4]:
print("\t>>>LOADING DATASETS")
dataset = None
classes = None

# Coordinate reference system. All datasets should be transformed to match this coordinate system.
AXIS_INSTANCE=0
AXIS_TIME=1
AXIS_DIMENSIONS=2

# Quaternion representation. All datasets should be transformed to match this quaternion representation.
# [qw, qi, qj qk]

	>>>LOADING DATASETS


In [5]:
# Load previously processed datasets
dataset_quaternion  =   utils.load_binaryfile_npy( gen_path_temp( FILENAME_DATASET_QUATERNION ) )
dataset_euler       =   utils.load_binaryfile_npy( gen_path_temp( FILENAME_DATASET_EULER ) )
dataset_yaw         =   utils.load_binaryfile_npy( gen_path_temp( FILENAME_DATASET_YAW ) )

# General variable to extract dataset stats
dataset=dataset_quaternion

In [6]:
if experiment_config.DATASET_MAIN == Datasets.Tsinghua:
        
    # Data for combined time series to cluster
    labels_filename = experiment_config.DATASET_LABELS # Cluster index TRUE_LABEL
    timestamps_filename = experiment_config.DATASET_TIMESTAMPS # Timestamps
    labels = pd.read_csv(labels_filename)
    timestamps = np.loadtxt(timestamps_filename)
    
    # # Classes are the labels of the videos 
    # classes = labels["videoId"].to_numpy(dtype=np.int32)
    # # Classes are the user watching the videos
    #classes = labels["user"].to_numpy(dtype=np.int32)

    classes = labels[experiment_config.CLASS_COLUMN_NAME].to_numpy(dtype=np.int32)


## Summary dataset

In [7]:
num_classes = np.unique(classes).size

if(dataset.ndim == 2):
    dataset = np.expand_dims(dataset, axis=2)

num_ts = dataset.shape[AXIS_INSTANCE]
length_ts = dataset.shape[AXIS_TIME]
num_dims = dataset.shape[AXIS_DIMENSIONS]

print("Timestamps:", type(timestamps), timestamps.shape)
print("Dataset", type(dataset), dataset.shape)
print("Classes", type(classes), classes.shape)
print(f"num_classes={num_classes}")
print(f"num_ts={num_ts}")
print(f"length_ts={length_ts}")
print(f"num_dims={num_dims}")

Timestamps: <class 'numpy.ndarray'> (3601,)
Dataset <class 'numpy.ndarray'> (432, 3601, 4)
Classes <class 'numpy.ndarray'> (432,)
num_classes=9
num_ts=432
length_ts=3601
num_dims=4


## `Summary`

Until this point, the head movements are stored as in these data representations: 
- Quaternion (`dataset_quaternion`)
- Euler (`dataset_euler`)
- Yaw (`dataset_yaw`)

## Differential Features

In [8]:
# Calculate velocity and acceleration
# according to formulas 14-17 in: doi.org/10.1007/s11045-018-0611-3
 
dataset_quat_vel = quaternion_math.quaternion_mult(dataset_quaternion[:,1:,:],quaternion_math.quaternion_conjugate(dataset_quaternion[:,:-1,:]))
dataset_quat_acc = quaternion_math.quaternion_mult(dataset_quat_vel[:,1:,:], quaternion_math.quaternion_conjugate(dataset_quat_vel[:,:-1,:]))

# dataset_quat_vel = dataset_quaternion[:,1:,:] - dataset_quaternion[:,:-1,:]
# dataset_quat_acc = dataset_quat_vel[:,1:,:] - dataset_quat_vel[:,:-1,:]

dataset_euler_vel = dataset_euler[:,1:,:] - dataset_euler[:,:-1,:]
dataset_euler_acc = dataset_euler_vel[:,1:,:] - dataset_euler_vel[:,:-1,:]

dataset_yaw_vel = dataset_yaw[:,1:,:] - dataset_yaw[:,:-1,:]
dataset_yaw_acc = dataset_yaw_vel[:,1:,:] - dataset_yaw_vel[:,:-1,:]

In [9]:
# Make all the arrays the same length, and reshape the timestamps accordingly
print(f"Timestamps: {timestamps.shape}")
print(f"Quat:  {dataset_quaternion.shape}, {dataset_quat_vel.shape}, {dataset_quat_acc.shape}")
print(f"Euler: {dataset_euler.shape}, {dataset_euler_vel.shape}, {dataset_euler_acc.shape}")
print(f"Yaw:   {dataset_yaw.shape}, {dataset_yaw_vel.shape}, {dataset_yaw_acc.shape}")

print(f">> Resizing!!")
diff_timestamps = timestamps[2:]

dataset_quaternion = dataset_quaternion[:,2:,:]
dataset_quat_vel = dataset_quat_vel[:,1:,:]

dataset_euler = dataset_euler[:,2:,:]
dataset_euler_vel = dataset_euler_vel[:,1:,:]

dataset_yaw = dataset_yaw[:,2:,:]
dataset_yaw_vel = dataset_yaw_vel[:,1:,:]

print(f"Timestamps: {diff_timestamps.shape}")
print(f"Quat:  {dataset_quaternion.shape}, {dataset_quat_vel.shape}, {dataset_quat_acc.shape}")
print(f"Euler: {dataset_euler.shape}, {dataset_euler_vel.shape}, {dataset_euler_acc.shape}")
print(f"Yaw:   {dataset_yaw.shape}, {dataset_yaw_vel.shape}, {dataset_yaw_acc.shape}")

Timestamps: (3601,)
Quat:  (432, 3601, 4), (432, 3600, 4), (432, 3599, 4)
Euler: (432, 3601, 3), (432, 3600, 3), (432, 3599, 3)
Yaw:   (432, 3601, 1), (432, 3600, 1), (432, 3599, 1)
>> Resizing!!
Timestamps: (3599,)
Quat:  (432, 3599, 4), (432, 3599, 4), (432, 3599, 4)
Euler: (432, 3599, 3), (432, 3599, 3), (432, 3599, 3)
Yaw:   (432, 3599, 1), (432, 3599, 1), (432, 3599, 1)


## Feature extraction

Multiple subexperiments:
- Quaternion: Raw, Vel, Acc
- Euler: Raw, Vel, Acc
- Yaw: Raw, Vel, Acc
- All: All Quaternion + All Euler (Yaw is included in Euler angles)

Each combination with feature-based classifiers:
- KNN
- DT
- RF
- GBM

In [10]:
print("\t>>>LOADING/CREATING TABULAR FEATURES")
# Filename of the file containing demographics and HMD movements data
features_quaternion_filename = gen_path_results("data_features_quat", subfolders="datasets_feature_based/", extension=".csv")
features_euler_filename      = gen_path_results("data_features_euler", subfolders="datasets_feature_based/", extension=".csv")
features_yaw_filename        = gen_path_results("data_features_yaw", subfolders="datasets_feature_based/", extension=".csv")
features_all_filename        = gen_path_results("data_features_all", subfolders="datasets_feature_based/", extension=".csv")

# Placeholders
features_quat  = None
features_euler = None
features_yaw   = None
features_all   = None

### INPUTS / OUTPUTS
"""EDIT CUSTOM FILENAMES"""
input_files = [features_quaternion_filename, features_euler_filename, features_yaw_filename, features_all_filename]

print(input_files)

RELOAD_TRIES = experiment_config.RELOAD_TRIES
# Try to load files maximum two times
for tries in range(RELOAD_TRIES):
    try:
        ### LOAD FILE
        print(f"Trying {tries+1}/{RELOAD_TRIES} to load files: {input_files}")
        
        ### CUSTOM SECTION TO READ FILES
        """EDIT CUSTOM READ"""
        features_quat = pd.read_csv(input_files[0]) # pd.DataFrame
        print(f"File {input_files[0]} was successfully loaded")
        features_euler = pd.read_csv(input_files[1]) # pd.DataFrame
        print(f"File {input_files[1]} was successfully loaded")
        features_yaw = pd.read_csv(input_files[2]) # pd.DataFrame
        print(f"File {input_files[2]} was successfully loaded")
        features_all = pd.read_csv(input_files[3]) # pd.DataFrame
        print(f"File {input_files[3]} was successfully loaded")

    except FileNotFoundError as e:
        ### CREATE FILE
        print(f"File not found. Creating again! {e}")

        ### CUSTOM SECTION TO CREATE FILES 
        """EDIT CUSTOM WRITE"""
        
        # Combine time series to extract joint features
        ARRAYS = [  np.concatenate([dataset_quaternion, dataset_quat_vel, dataset_quat_acc], axis=AXIS_DIMENSIONS), \
                    np.concatenate([dataset_euler, dataset_euler_vel, dataset_euler_acc], axis=AXIS_DIMENSIONS), \
                    np.concatenate([dataset_yaw, dataset_yaw_vel, dataset_yaw_acc], axis=AXIS_DIMENSIONS), \
                    np.concatenate([dataset_quaternion, dataset_quat_vel, dataset_quat_acc,                      
                                        dataset_euler, dataset_euler_vel, dataset_euler_acc], axis=AXIS_DIMENSIONS)
                 ]

        HEADERS = [ HEADER_QUAT, HEADER_EULER, HEADER_YAW, HEADER_ALL]

        for idx in range(len(ARRAYS)):
            array_to_process = ARRAYS[idx]
            header_to_process = HEADERS[idx]
            data_feat_array, colnames = ts_processing.extract_summary_statistics_with_non_overlapping_time_window(array_to_process, \
                                                                diff_timestamps, \
                                                                class_labels=classes, \
                                                                time_limit_secs=(timestamps[0],timestamps[-1]), \
                                                                window_width_secs=SLIDING_WINDOW_WIDTH_SECS, \
                                                                dim_names = header_to_process)
            dataframe_features = pd.DataFrame(data=data_feat_array, columns=colnames)
            for i in [1, 2, -1]:
                dataframe_features[colnames[i]] = dataframe_features[colnames[i]].astype(int)
            dataframe_features.to_csv(input_files[idx], index=False)

        ### ---- CONTROL RETRIES
        if tries+1 < RELOAD_TRIES:
            continue
        else:
            raise
    break

	>>>LOADING/CREATING TABULAR FEATURES
['./results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_quat.csv', './results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_euler.csv', './results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_yaw.csv', './results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_all.csv']
Trying 1/2 to load files: ['./results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_quat.csv', './results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_euler.csv', './results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_yaw.csv', './results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_all.csv']
File ./results/Tsinghua/2_FeatureBasedClassifiers/datasets_feature_based/data_features_quat.csv was successfully loaded
File ./results/Tsinghua/2_FeatureBasedClassifiers/datasets_featu

## `NOTE:` REDEFINITION OF DATASETS
The dictionary is redefined now, since the datasets were properly loaded/created

In [11]:
NUM_CLASSES = num_classes # Global variable counting number of different classes in the dataset
# REDEFINE DICT WITH CORRESPONDING DATASETS
DICT_DATA = {
    DataRepresentation.Quaternion:  features_quat.copy(),
    DataRepresentation.Euler:       features_euler.copy(),
    DataRepresentation.Yaw:         features_yaw.copy(),
    DataRepresentation.All:         features_all.copy()
}

for dr, _data in DICT_DATA.items():
    print(f"Dataset '{dr}' shape: \t{_data.shape}")

Dataset 'quaternion' shape: 	(51840, 64)
Dataset 'euler' shape: 	(51840, 49)
Dataset 'yaw' shape: 	(51840, 19)
Dataset 'all' shape: 	(51840, 109)


---
## Feature-based Classification

In [12]:
# Contains the dataframe with compiled results
classif_results = None

In [13]:
print("\t>>>LOADING/CREATING CLASSIFICATION RESULTS")
# Filename of the file containing demographics and HMD movements data
classification_results_filename = gen_path_results(experiment_config.RESULTS_FILENAME, extension=".csv")

### INPUTS / OUTPUTS
"""EDIT CUSTOM FILENAMES"""
input_files = [classification_results_filename]

RELOAD_TRIES = experiment_config.RELOAD_TRIES
# Try to load files maximum two times
for tries in range(RELOAD_TRIES):
    try:
        ### LOAD FILE
        print(f"Trying {tries+1}/{RELOAD_TRIES} to load files: {input_files}")
        
        ### CUSTOM SECTION TO READ FILES
        """EDIT CUSTOM READ"""
        classif_results = pd.read_csv(input_files[0]) # pd.DataFrame
        print(f"File {input_files[0]} was successfully loaded")

    except FileNotFoundError as e:
        ### CREATE FILE
        print(f"File not found. Creating again! {e}")

        ### CUSTOM SECTION TO CREATE FILES 
        """EDIT CUSTOM WRITE"""
        
        idx = 1 # Iterator for row index
        total_iter = len(DICT_DATA.keys()) * len(DICT_CLASSIFIERS.keys()) # How many total iterations are going to be conducted
        print(f"Iterating {total_iter} times")
        # f = IntProgress(min=0, max=classif_results.shape[0])
        # display(f)

        ### Iterate over datasets and classifiers

        # For each dataset
        for iter_datarep, iter_data in DICT_DATA.items():
            # Apply each classifier model
            for iter_classifier,iter_model in DICT_CLASSIFIERS.items():
                
                # Print messages
                if idx%experiment_config.DISPLAY_ITER_STEP==0: 
                    print(f" | Iteration {idx}/{total_iter} \t> dataset: {str(iter_datarep)} \tclassifier: {str(iter_classifier)}")
                idx = idx + 1

                # Separate training and testing sets
                iter_datadata = iter_data.drop(['instanceId','timeId','timestamp'], axis='columns')
                X = iter_data.drop(["class"],axis="columns")
                y = iter_data["class"]

                # Standardize (z-norm) if model is KNN
                if iter_model is DICT_CLASSIFIERS[Classifiers.KNN]:
                    X = StandardScaler().fit_transform(X)

                # Cross-validation (CV)
                cv_results =  cross_validate(iter_model, X, y, cv=strat_KFold, scoring=SCORING_METRICS, n_jobs=experiment_config.N_JOBS_PARALLEL, verbose=3) # 

                # Add more information to CV results
                cv_results[experiment_config.COLUMNS_LABELS[0]] = [ str(iter_datarep) ] * N_SPLITS_CV # Datarep
                cv_results[experiment_config.COLUMNS_LABELS[1]] = [ str(iter_classifier) ] * N_SPLITS_CV # Classifier
                cv_results[experiment_config.COLUMNS_LABELS[2]] = np.arange(N_SPLITS_CV) # Fold
                # for k,v in cv_results.items():
                    # print(f"{k}: \t{v}")

                # From dict to pandas DataFrame
                iteration_results = pd.DataFrame(cv_results.copy(), columns=sorted(cv_results.keys()))

                # Extend dataframe
                if classif_results is None:
                    classif_results = iteration_results
                else:
                    classif_results = classif_results.append(iteration_results, ignore_index=True)

                # END: Classifiers

            # } END: Dataset
            print(f"\t\t>> FINISHED (dataset,distMetr)")
        # } END: Loop done

        # for i in [0,1,2,3]:
            # classif_results[experiment_config.COLUMNS_LABELS[i]] = classif_results[experiment_config.COLUMNS_LABELS[i]].astype(int)

        # Save files
        classif_results.to_csv(input_files[0], index=False)

        ### ---- CONTROL RETRIES
        if tries+1 < RELOAD_TRIES:
            continue
        else:
            raise
    break

	>>>LOADING/CREATING CLASSIFICATION RESULTS
Trying 1/2 to load files: ['./results/Tsinghua/2_FeatureBasedClassifiers/classif_results.csv']
File not found. Creating again! [Errno 2] No such file or directory: './results/Tsinghua/2_FeatureBasedClassifiers/classif_results.csv'
Iterating 20 times
 | Iteration 1/20 	> dataset: quaternion 	classifier: Classifiers.MLP


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed:  2.1min remaining: 11.8min
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed:  2.3min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  3.1min remaining:   32.6s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.3min finished


 | Iteration 2/20 	> dataset: quaternion 	classifier: Classifiers.KNN


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed:    5.0s remaining:   28.8s
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed:    5.6s remaining:    5.6s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:   10.9s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   11.3s finished


 | Iteration 3/20 	> dataset: quaternion 	classifier: Classifiers.DT


...

 | Iteration 19/20 	> dataset: all 	classifier: Classifiers.RF


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed:  2.1min remaining: 12.2min
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed:  2.1min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  3.5min remaining:   36.6s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.5min finished


 | Iteration 20/20 	> dataset: all 	classifier: Classifiers.GBM


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  20 | elapsed: 21.9min remaining: 123.9min
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed: 21.9min remaining: 21.9min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 40.1min remaining:  7.1min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 40.1min finished


		>> FINISHED (dataset,distMetr)
Trying 2/2 to load files: ['./results/Tsinghua/2_FeatureBasedClassifiers/classif_results.csv']
File ./results/Tsinghua/2_FeatureBasedClassifiers/classif_results.csv was successfully loaded


In [14]:
print(">> FINISHED WITHOUT ERRORS!!")

>> FINISHED WITHOUT ERRORS!!
